In [ ]:
import pymongo
from pprint import pprint

## Database Connection

In [ ]:
db = pymongo.MongoClient()

## Inspect

In [ ]:
db.database_names()

In [ ]:
db['test'].collection_names()

In [ ]:
# db.collection.findOne()
pprint(
    db['test']['zips'].find_one()
)

In [ ]:
# db.collection.count()
db['test']['zips'].count()

## Aggregation Pipeline

### stage operators I

In [ ]:
pipeline = [
    {"$match": {"city": "NEW YORK"}},
]

result = db['test']['zips'].aggregate(pipeline)
pprint(list(result)[:5])

In [ ]:
pipeline = [
    {"$match": {"city": "NEW YORK"}},
    {"$project": {"city": 1, "_id": 1}}
]

result = db['test']['zips'].aggregate(pipeline)
pprint(list(result)[:3])

### stage operators II

In [ ]:
pipeline = [
    {"$match": {"city": "NEW YORK"}},
    {"$project": {"city": 1, "_id": 1}},
    {"$sort": {'_id': -1}}
]

result = db['test']['zips'].aggregate(pipeline)
pprint(list(result)[:3])

In [ ]:
from bson.son import SON
pipeline = [
    {"$match": {"city": "NEW YORK"}},
    {"$project": {"city": 1, "_id": 1, "pop": 1}},
    # python only
    {"$sort": SON([("_id", 1),("pop", -1)])}
#    {"$sort": SON([("pop", -1), ("_id", 1)])}
]

result = db['test']['zips'].aggregate(pipeline)
pprint(list(result)[:3])

In [ ]:
pipeline = [
    {"$match": {"city": "NEW YORK"}},
    {"$project": {"city": 1, "_id": 1}},
    {"$limit": 2}
]

result = db['test']['zips'].aggregate(pipeline)
pprint(list(result))

In [ ]:
pipeline = [
    {"$match": {"city": "NEW YORK"}},
    {"$project": {"city": 1, "_id": 1}},
    {"$skip": 18},
    {"$limit": 2},
]

result = db['test']['zips'].aggregate(pipeline)
pprint(list(result))

### stage operators III

In [ ]:
pipeline = [
    {"$group": {"_id": "$state",
               "totalPop": {"$sum": "$pop"}}},
    {"$match": {"totalPop": {"$gt": 10000000}}},
]

result = db['test']['zips'].aggregate(pipeline)
pprint(list(result))

In [ ]:
"""
SELECT state, SUM(pop) AS totalPop
FROM zipcodes
GROUP BY state
HAVING totalPop >= (10000000)
"""

In [ ]:
pipeline = [
    {"$group": {"_id": "$state",
               "totalPop": {"$sum": "$pop"}}},
    {"$group": {"_id": None,
               "avgPop": {"$avg": "$totalPop"}}}
]

result = db['test']['zips'].aggregate(pipeline)
pprint(list(result))

In [ ]:
pipeline = [
    {"$group": {"_id": {"state": "$state", "city": "$city"},
               "totalCityPop": {"$sum": "$pop"}}},
    {"$group": {"_id": "$_id.city",
               "avgCityPop": {"$avg": "$totalCityPop"}}},
    {"$sort": {"avgCityPop": -1}}
]

result = db['test']['zips'].aggregate(pipeline)
pprint(list(result)[:5])